In [10]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, TrailingStrategy
import yfinance as yf
import ta

In [11]:
ticker = 'BNB-USD'
start_date = '2017-09-14'
end_date = '2023-05-29'

def load_data(ticker):
    try:            
      data = yf.download(ticker, start_date, end_date)
      data.reset_index(inplace=True)
      return data
    except Exception as e:
      print(f'An exception occurred while executing load_data: {e}')

In [12]:
def RsiIndicator(close, window):
    rsi = ta.momentum.RSIIndicator(close, window=window)
    return rsi.rsi()

class RsiOscillator(TrailingStrategy):
    """
    Buy when we cross the lower bound to the upside
    hold until we get stopped out
    """

    rsi_window = 14
    lower_bound = 70
    stop_range = 10

    def init(self):
        super().init()
        super().set_trailing_sl(self.stop_range)
        self.rsi = self.I(RsiIndicator, self.data.Close.s, self.rsi_window)

    def next(self):
        super().next()

        if len(self.trades) == 0:
            if crossover(self.rsi, self.lower_bound):
                self.buy()

my_data = load_data(ticker)

bt = Backtest(my_data, RsiOscillator, cash = 10000, commission=0.07)

stats = bt.run(
        rsi_window = 21,
        lower_bound = 40,
        stop_range = 5,
        )

print(stats)
bt.plot()

[*********************100%***********************]  1 of 1 completed
Start                                     0.0
End                                    2026.0
Duration                               2026.0
Exposure Time [%]                   69.166256
Equity Final [$]                  8694.592267
Equity Peak [$]                  38837.283628
Return [%]                         -13.054077
Buy & Hold Return [%]            15686.356678
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -80.922815
Avg. Drawdown [%]                  -19.698219
Max. Drawdown Duration                 1499.0
Avg. Drawdown Duration             128.333333
# Trades                                 17.0
Win Rate [%]                        35.294118
Best Trade [%]                     283.609411
Worst Trade

/var/folders/69/k2882s3d5jbbkmnl3p42wg4ws68910/T/ipykernel_14219/1195659991.py:29: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(my_data, RsiOscillator, cash = 10000, commission=0.07)


Row(id='3799', ...)